In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import glob
import numpy as np

In [3]:
# function to detect human face using OpenCV Haar-Cascade Classifier
def haar_cascade_ADB(image):
    """
    This function takes an image and detect faces.
    Return: detected faces - array([x_top_left, y_top_left, width, height], ....)
            containing bounding boxes for detected faces.
    """

    # convert image to grayscale
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # initialize haar cascade model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')

    # detect faces
    detected_faces = face_cascade.detectMultiScale(img_gray)

    return detected_faces

In [4]:
# function to create detection file for each image
def create_detection_file(img_name, detected_faces, out_dir):
    """
        Input: detected faces - array([x_top_left, y_top_left, width, height], ....)
               containing bounding boxes for detected faces

        Returns: img_name.txt, in which each line has the following format
                 <class_name> <confidence> <Top_left_x> <Top_left_y> <Bottom_right_x> <Bottom_right_y>
    """
    class_name = 'face'
    conf_score = 1.0
    bbox_lists = []
    for bbox in detected_faces:
        print(bbox, type(bbox))
        top_left_x, top_left_y, w, h = bbox.tolist()

        bottom_right_x = top_left_x + w 
        bottom_right_y = top_left_y + h

        bbox_lists.append([class_name, conf_score, top_left_x, top_left_y, bottom_right_x, bottom_right_y])

    with open(out_dir+'/'+img_name+'.txt', 'w') as f:
        for bbox in bbox_lists:
            for i in bbox:
                f.write(str(i)+' ')
            f.write('\n')
    f.close()

In [5]:
wider_val_imgs = '/content/drive/MyDrive/Predective_dataset/WIDER_val/WIDER_val/images'
pred_out_dir = '/content/drive/MyDrive/Predective_dataset/det_files'

folders = glob.glob(wider_val_imgs+'/*')

for folder in folders:
    img_paths = glob.glob(folder+'/*')

    for path in img_paths:
        img_name = path.split('/')[-1].split('.')[0]

        img = cv2.imread(path)    
        faces = haar_cascade_ADB(img)
        create_detection_file(img_name, faces, pred_out_dir)


Streaming output truncated to the last 5000 lines.
[671 555  40  40] <class 'numpy.ndarray'>
[541 269 198 198] <class 'numpy.ndarray'>
[138 340 241 241] <class 'numpy.ndarray'>
[ 763 1098   81   81] <class 'numpy.ndarray'>
[534 289  42  42] <class 'numpy.ndarray'>
[ 58 300  38  38] <class 'numpy.ndarray'>
[ 511 1058   42   42] <class 'numpy.ndarray'>
[ 623 1064   36   36] <class 'numpy.ndarray'>
[ 422 1032   45   45] <class 'numpy.ndarray'>
[ 445 1065   63   63] <class 'numpy.ndarray'>
[ 280 1076   60   60] <class 'numpy.ndarray'>
[ 721 1104   67   67] <class 'numpy.ndarray'>
[385 225  28  28] <class 'numpy.ndarray'>
[275 221  46  46] <class 'numpy.ndarray'>
[ 61 266  49  49] <class 'numpy.ndarray'>
[782 384  53  53] <class 'numpy.ndarray'>
[830 221 104 104] <class 'numpy.ndarray'>
[580 555  29  29] <class 'numpy.ndarray'>
[469 572  38  38] <class 'numpy.ndarray'>
[615 590  41  41] <class 'numpy.ndarray'>
[853 598  71  71] <class 'numpy.ndarray'>
[507 596  57  57] <class 'numpy.ndarray

In [6]:
# function to parse WIDERFace annotations and create a dictionary
def parse_wider_annots(annots_file):
    """
          Input: WIDER-format annotation file:
            - first line is image file name
            - second line is an integer, for `n` faces in that image
            - next `n` lines are bounding box(bbox) coordinates
            - again, next line is image file name
            - bbox are [top_left_x top_left_y w h blur expression illumination invalid occlusion pose]
          Returns a dict: {'img_filename': [[x,y,w,h], [x,y,w,h], ....]}
        """

    # read annotation file
    ann_file = open(annots_file, 'r')

    is_img = True          # a line has image name or not
    isNum_annots = False   # a line has number of annotations or not
    start_annots_count = False
    annots_count = 0
    num_annots = -1

    # initialize dictionary to add image paths and their corresponding annotations
    ann_dict = {}
    img_name = ''     # initialize empty image name

    for line in ann_file:
        line = line.strip()
        
        if line == '0 0 0 0 0 0 0 0 0 0':
            if annots_count == num_annots + 1:
                start_annots_count = False
                annots_count = 0
                is_img = True  # next line is image file
                isNum_annots = False
                num_annots = -1
                ann_dict.pop(img_name)
            continue

        if is_img:
            # current line has image name
            is_img = False     # false until next line comes with image name
            isNum_annots = True
            img_name = line
            ann_dict[img_name] = []  # creating a key in dictionary for current image name
            continue

        if isNum_annots:
            # get number of annotations
            num_annots = int(line)
            isNum_annots = False     # false until reach to next image annotations
            if num_annots > 0:
                start_annots_count = True  # start counting annotations
                annots_count = 0
            else:
                # no annotations for this image
                is_img = True     # next line will be the next image name
                num_annots = -1   # will count again
            continue

        if start_annots_count:
            # After the line with # annotations, annotations: [x,y,w,h,......] start
            annots = [float(x) for x in line.split()]  # split on whitespace
            ann_dict[img_name].append(annots[:4])      # add annotations:[x,y,w,h] for current image
            annots_count += 1

        if annots_count == num_annots:
            # gone through all the annotations for current image
            start_annots_count = False
            annots_count = 0
            is_img = True        # next line will be image name
            isNum_annots = False
            num_annots = -1

    return ann_dict

In [7]:
#  function to create ground truth bbox files
def create_gt_file(ann_dict, out_dir):
    for key, values in ann_dict.items():
        file = key.split('/')[-1]
        img_name = file.split('.')[0]

        class_name = 'face'
        conf_score = 1.0
        bbox_lists = []
        for bbox in values:
            top_left_x, top_left_y, w, h = bbox

            bottom_right_x = top_left_x + w 
            bottom_right_y = top_left_y + h

            bbox_lists.append([class_name, conf_score, top_left_x, top_left_y, bottom_right_x, bottom_right_y])

        with open(out_dir+'/'+img_name+'.txt', 'w') as f:
            for bbox in bbox_lists:
                for i in bbox:
                    f.write(str(i)+' ')
                f.write('\n')
        f.close()

In [10]:
wider_val_gt_ann = '/content/drive/MyDrive/Predective_dataset/wider_face_split/wider_face_split/wider_face_val_bbx_gt.txt'
gt_out_dir = '/content/drive/MyDrive/Predective_dataset/gt_files'
ann_dict = parse_wider_annots(wider_val_gt_ann)

create_gt_file(ann_dict, gt_out_dir)